 This Python 3 environment comes with many helpful analytics libraries installed
 It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
 For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

 Input data files are available in the read-only "../input/" directory
 For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

 You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"  You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load Libraries

In [1]:
import numpy as np 
import pandas as pd
import math

import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import DataLoader, Dataset
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

from sklearn.model_selection  import train_test_split
from sklearn.model_selection import KFold


from transformers import AutoModel, AutoTokenizer, BertTokenizer, AutoConfig

import itertools
import gc
import os 
import random

import spacy
import time
import timeit

In [2]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(seed=1326)

# HyperParameters

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 16
epochs = 3  # The number of epochs
embedding_dim = 300
MODEL_NAME =  '../input/huggingface-roberta-variants/roberta-base/roberta-base'
# MODEL_NAME2 =  '../input/huggingface-roberta-variants/roberta-base/roberta-base'

# ../input/huggingface-bert/bert-base-cased
MODEL_NAME2 = None
useFeatures = False

# load dataframe

In [4]:
datadir = '/kaggle/input/commonlitreadabilityprize'
testdir = datadir + '/test.csv'
test_df = pd.read_csv(testdir)

test_df.head(5)

,id,url_legal,license,excerpt
0,c0f722661,NaN,NaN,My hope lay in Jack's promise that he would ke...
1,f0953f0a5,NaN,NaN,Dotty continued to go to Mrs. Gray's every nig...
2,0df072751,NaN,NaN,It was a bright and cheerful scene that greete...
3,04caf4e0c,https://en.wikipedia.org/wiki/Cell_division,CC BY-SA 3.0,Cell division is the process by which a parent...
4,0e63f8bea,https://en.wikipedia.org/wiki/Debugging,CC BY-SA 3.0,Debugging is the process of finding and resolv...


In [5]:
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print(method.__name__, (te - ts) * 1000)
        return result
    return timed

In [6]:
# Readability Features


# Splits the text into sentences, using
# Spacy's sentence segmentation which can
# be found at https://spacy.io/usage/spacy-101
def break_sentences(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    return list(doc.sents)
 
# Returns Number of Words in the text
def word_count(sentences):
    words = 0
    for sentence in sentences:
        words += len([token for token in sentence])
    return words
 
# Returns the number of sentences in the text
def sentence_count(sentences):
    return len(sentences)
 
# Returns average sentence length
def avg_sentence_length(sentences, words):
    average_length = float(words / sentences)
    return average_length
 
# Textstat is a python package, to calculate statistics from
# text to determine readability,
# complexity and grade level of a particular corpus.
# Package can be found at https://pypi.python.org/pypi/textstat
def syllables_count(word):
    word = str(word)
    syllable_count = 0
    vowels = 'aeiouy'
    if word[0] in vowels:
        syllable_count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            syllable_count += 1
    if word.endswith('e'):
        syllable_count -= 1
    if word.endswith('le') and len(word) > 2 and word[-3] not in vowels:
        syllable_count += 1
    if syllable_count == 0:
        syllable_count += 1
    return syllable_count
 
# Returns the average number of syllables per
# word in the text
def avg_syllables_per_word(sentences, words):
    syllable = 0
    for word in sentences:
        syllable += syllables_count(word)
    ASPW = float(syllable) / float(words)
    return round(ASPW, 1)
 
# Return total Difficult Words in a text
def difficult_words(sentences, text):
     
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    # Find all words in the text
    words = []
    for sentence in sentences:
        words += [str(token) for token in sentence]
 
    # difficult words are those with syllables >= 2
    # easy_word_set is provide by Textstat as
    # a list of common words
    diff_words_set = set()
     
    for word in words:
        syllable_count = syllables_count(word)
        if word not in nlp.Defaults.stop_words and syllable_count >= 2:
            diff_words_set.add(word)
 
    return len(diff_words_set)
 


def flesch_reading_ease(avg_length , avg_syllables ):
    """
        Implements Flesch Formula:
        Reading Ease score = 206.835 - (1.015 × ASL) - (84.6 × ASW)
        Here,
          ASL = average sentence length (number of words
                divided by number of sentences)
          ASW = average word length in syllables (number of syllables
                divided by number of words)
    """
    FRE = 206.835 - float(1.015 * avg_length) -\
          float(84.6 * avg_syllables)
    return round(FRE, 2)
 
 
def gunning_fog(avgLength, diff, words):
    per_diff_words = (diff / words * 100) + 5
    grade = 0.4 * (avgLength + per_diff_words)
    return grade
 
 
def dale_chall_readability_score(words, avg_length, diff):
    """
        Implements Dale Challe Formula:
        Raw score = 0.1579*(PDW) + 0.0496*(ASL) + 3.6365
        Here,
            PDW = Percentage of difficult words.
            ASL = Average sentence length
    """
    # Number of words not termed as difficult words
    count = words - diff
    if words > 0:
 
        # Percentage of words not on difficult word list
 
        per = float(count) / float(words) * 100
     
    # diff_words stores percentage of difficult words
    diff_words = 100 - per
 
    raw_score = (0.1579 * diff_words) + \
                (0.0496 * avg_length)
     
    # If Percentage of Difficult Words is greater than 5 %, then;
    # Adjusted Score = Raw Score + 3.6365,
    # otherwise Adjusted Score = Raw Score
 
    if diff_words > 5:      
 
        raw_score += 3.6365
         
    return round(raw_score, 2)



def flesch_reading_age(ASL, ASW):
    """
        Implements Flesch-Kincaid Grade Level Formula:
        Reading Ease score = (0.39 x ASL) + (11.8 x ASW) - 15.59
        Here,
          ASL = average sentence length (number of words
                divided by number of sentences)
          ASW = average word length in syllables (number of syllables
                divided by number of words)
                
    """
    
    FKRA = (0.39 * ASL) + (11.8 * ASW) - 15.59 
    return FKRA


@timeit
def calculateFeatures(df):
    
    flesch_reading = []
    flesch_age = []
    gunning = []
    dale_chall = []
    wordCount = []
    avgSentenceLength = []
    avgSyllable = []
    
    for text in df['excerpt']:
        sentences = break_sentences(text)
        word_Count = word_count(sentences)
        avgLength = avg_sentence_length(len(sentences), word_Count)
        diffWords = difficult_words(sentences, text)
        avgSyllables = avg_syllables_per_word(sentences, word_Count)
        
        wordCount.append(word_Count)
        avgSentenceLength.append(avgLength)
        avgSyllable.append(avgSyllables)
        dale_chall.append(dale_chall_readability_score(word_Count,avgSyllables , diffWords ))
        flesch_age.append(flesch_reading_age(avgLength,  avgSyllables ))
        flesch_reading.append(flesch_reading_ease(avgLength , avgSyllables))
        gunning.append(gunning_fog(avgLength, diffWords, word_Count))

    df['flesch_age'] = flesch_age
    df['flesch_reading'] = flesch_reading
    df['wordCount'] = wordCount
    df['avgSentenceLength'] = avgSentenceLength
    df['avgSentenceSyllable'] = avgSyllable
    df['dale_chall'] = dale_chall
    df['gunning_fog'] = gunning


    
test_df = test_df[['id','excerpt']]

if useFeatures:
    calculateFeatures(test_df)

    
    
test_df.head(5)

,id,excerpt
0,c0f722661,My hope lay in Jack's promise that he would ke...
1,f0953f0a5,Dotty continued to go to Mrs. Gray's every nig...
2,0df072751,It was a bright and cheerful scene that greete...
3,04caf4e0c,Cell division is the process by which a parent...
4,0e63f8bea,Debugging is the process of finding and resolv...


# CommonLitDataset

In [7]:

class TestDataset(Dataset):
    def __init__(self, df):
        self.df = df 
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        text = self.df.iloc[idx][1]     
        if useFeatures:
            readability = np.array([x for x in self.df.iloc[idx][2:].values])
        else:
            readability = torch.zeros((1,7))
        return text, readability
    
    
test_data = TestDataset(test_df)
test_dataloader = DataLoader(test_data, batch_size = 2, shuffle = False)

# Transformer Model

In [8]:
class CommonLitModel(nn.Module):
    def __init__(self,path, path2 = None):
        super(CommonLitModel,self).__init__()
        self.config = AutoConfig.from_pretrained(path)
        self.config.update({'output_hidden_states':True})
        self.bert = AutoModel.from_pretrained(path,output_hidden_states=False)
        
        if path2:
            self.bert2 = AutoModel.from_pretrained(path2, output_hidden_states = False) 
            self.linear1 = nn.Linear(1536,1536)
            self.linear2 = nn.Linear(1536,1)
        else:
            print('768 Features used')
            self.linear1 = nn.Linear(768,768)
            self.linear2 = nn.Linear(768,1)
            
            
        self.linear = nn.Linear(775,1)   
        self.dropout = nn.Dropout(0.50)
        self.lrelu = nn.LeakyReLU()
    

    def forward(self,xb, x2 = None, readability = None):
        x = self.bert(**xb)[1]
        if x2:
            x1 = self.bert2(**x2)[1]
            x = torch.cat((x, x1))
#             x = torch.mean(torch.stack([x, x1]))
        
#         x = self.dropout(x)
#         x = self.linear1(x)
#         x = self.lrelu(x) 
#         print(x.size())
#         print(readability.size())
        if useFeatures:
            x = torch.cat((x,readability),1)
            x = self.dropout(x)
            x = self.linear(x)
        else:
            x = self.dropout(x)
            x = self.linear2(x)
        return x
    
    

# model = CommonLitModel(MODEL_NAME,MODEL_NAME2).to(device)
# torch.save(model.state_dict(), 'initialModel')

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if MODEL_NAME2:
    tokenizer2 = AutoTokenizer.from_pretrained(MODEL_NAME2)

# Inference

In [9]:
model = torch.load('../input/commonlitmodelexperiments/bestModel').eval()

model.eval()

submission = test_df

predictions = []

for data, readability in test_dataloader:
    tokens  = tokenizer.batch_encode_plus([*data],pad_to_max_length='max_length', return_tensors='pt').to(device)
    readability = torch.tensor(readability).float().to(device)
    if MODEL_NAME2:
        tokens2  = tokenizer2.batch_encode_plus([*data],pad_to_max_length='max_length', return_tensors='pt').to(device)
        predictions += [float(pred[0]) for pred in model(tokens,tokens2, readability)] 
    else:
        predictions += [float(pred[0]) for pred in model(tokens, None, readability)]
    
    
submission['target'] = predictions
submission = submission[['id', 'target']]

submission.to_csv('submission.csv', index=False)
submission

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


,id,target
0,c0f722661,-0.504835
1,f0953f0a5,-0.388398
2,0df072751,-0.202327
3,04caf4e0c,-2.591927
4,0e63f8bea,-2.026085
5,12537fe78,-1.145287
6,965e592c0,0.639455
